In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from time import sleep

driver = webdriver.Chrome()
driver.set_window_size(1038, 806)
driver.set_window_position(0, 200)
driver.get("http://www.twitter.com")

In [15]:
# Function of waiting until the present of the element on the web page
def waiting_func(by_variable, attribute):
    try:
        WebDriverWait(driver, 20).until(lambda x: x.find_element(by=by_variable,  value=attribute))
    except (NoSuchElementException, TimeoutException):
        print('{} {} not found'.format(by_variable, attribute))
        exit()

In [16]:
waiting_func(By.NAME, "session[username_or_email]")

In [17]:
email = driver.find_element_by_name('session[username_or_email]')

In [18]:
print(email)

<selenium.webdriver.remote.webelement.WebElement (session="93b8e93aeb748e05d109a449ae1a98c6", element="f34b12ce-784a-4046-a30a-59797ed30120")>


In [19]:
email.send_keys('PranavKasetti')

In [20]:
password = driver.find_element_by_name('session[password]')
password.send_keys('', Keys.ENTER)

In [21]:
waiting_func('css selector', "[aria-label=Profile]")
profile = driver.find_element_by_css_selector("a[aria-label='Profile']")

In [22]:
profile.click()

In [23]:
paths = []
for _ in range(2):
    waiting_func('css selector', "a[aria-label='View Tweet activity']")
    last_height = driver.execute_script("return document.body.scrollHeight")
    traffic_path = driver.find_elements_by_css_selector("a[aria-label='View Tweet activity']")
    paths.extend([traffic.get_attribute('href') for traffic in traffic_path])
    driver.execute_script("window.scrollTo(0, {})".format(last_height+500))
    sleep(3)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if last_height == new_height:
        break

In [24]:
import pickle

def open_url(driver, url):
    """Open a URL using the driver's base URL"""
    cookies = driver.get_cookies()

    dummy_url = '/404error/wtheck'
    # Navigate to a dummy url on the same domain.
    driver.get("https://www.twitter.com" + dummy_url)

    driver.get(url)
    for cookie in cookies:
        new_cook = { 'name': cookie['name'], 'value': cookie['value'] }
        driver.add_cookie(new_cook)
    driver.refresh()
    sleep(0.5)

In [27]:
def analyse(path):
    open_url(driver, path)
    waiting_func(By.TAG_NAME, 'iframe')
    iframe = driver.find_element_by_tag_name('iframe')
    driver.switch_to.frame(iframe)
    waiting_func(By.TAG_NAME, 'body')
    detail = driver.find_element_by_tag_name('body')
    try:
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'ep-MetricTopContainer')))
    except TimeoutException as e:
        print("Wait Timed out")
        print(e)
    impression = detail.find_element_by_class_name('ep-MetricTopContainer')
    print(impression.text)
    try:
        WebDriverWait(driver, 3).until(
            lambda x: x.find_element(by='class name', value='ep-ViewAllEngagementsButton'))
    except TimeoutException:
        pass
    view_all = driver.find_element_by_class_name('ep-ViewAllEngagementsButton')
    view_all.click()
    waiting_func('class name', 'ep-EngagementsSection')
    engagesection = driver.find_element_by_class_name('ep-EngagementsSection')
    waiting_func('class name', 'ep-MetricTopContainer')
    engagement = engagesection.find_element_by_class_name('ep-MetricTopContainer')
    print(engagement.text)
    waiting_func('class name', 'ep-DetailedEngagementsSection')
    detail = engagesection.find_element_by_class_name('ep-DetailedEngagementsSection')
    waiting_func('class name', 'ep-SubSection')
    engagement_details = detail.find_elements_by_class_name('ep-SubSection')
    for _ in engagement_details:
        print(_.text)

In [29]:
for path in paths:
    analyse(path)

Impressions
132
Total engagements
8
Profile clicks
4
number of clicks on your name, @handle, or profile photo
Likes
2
times people liked this Tweet
Follows
1
number of people who followed you directly from this Tweet
Detail expands
1
times people viewed the details about this Tweet
Impressions
175
Total engagements
16
Profile clicks
11
number of clicks on your name, @handle, or profile photo
Likes
4
times people liked this Tweet
Detail expands
1
times people viewed the details about this Tweet
Impressions
311
Total engagements
25
Profile clicks
10
number of clicks on your name, @handle, or profile photo
Media engagements
9
number of clicks on your media counted across videos, vines, gifs, and images
Link clicks
3
clicks on a URL or Card in this Tweet
Likes
2
times people liked this Tweet
Replies
1
replies to this Tweet
Impressions
467
Total engagements
19
Media engagements
10
number of clicks on your media counted across videos, vines, gifs, and images
Profile clicks
5
number of clicks